In [1]:
# This is the code for SI 671 Kaggle competition 1

In [2]:
import pandas as pd
import numpy as np


In [3]:
# !kaggle competitions download -c f-25-si-670-kaggle-1

In [4]:
kaggle_train = pd.read_csv('si670_kaggle1_train.csv', index_col=2)
kaggle_train.head()

,text,label
id,,
0,White girls very rarely date Asian men. Even i...,1
1,I am a 23 year old male Indian American male. ...,1
2,"Take three people, Persons A, B, and C. They l...",1
3,(A) Work part-time in high school; Then go to ...,1
4,When police introduce a new form of speed prev...,1


In [5]:
kaggle_validation = pd.read_csv('si670_kaggle1_validation.csv', index_col=2)
kaggle_validation.head()

,text,label
id,,
319071,"In much the same way as drugs, prostitution, g...",1
319072,A friend and I were talking about the current ...,1
319073,While I believe this is primarily an American ...,1
319074,The basis of this viewpoint is that a system t...,1
319075,"Ah, like it says in ""Dust In The Wind."" Every ...",1


In [20]:
kaggle_test = pd.read_csv('test.csv')
kaggle_test.tail(1)

,id,text
60742,436605,The effects on miRNAs caused by HIVSIV infecti...


In [ ]:
# first, a baseline model with Hashing Vectorizer and SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import HashingVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier

X_train, X_test, y_train, y_test = train_test_split(
    kaggle_train['text'], kaggle_train['label'], test_size=0.2, random_state=42)

pipe = Pipeline([
    ("hash", HashingVectorizer(analyzer="word", ngram_range=(1,2),
                               n_features=2**21, alternate_sign=False,
                               norm=None, dtype="float32")),
    ("tfidf", TfidfTransformer()),  # produces float64 idf, OK; keeps sparse
    ("clf", SGDClassifier(loss="log_loss", alpha=1e-4,
                          early_stopping=True, n_jobs=1))
], memory="tmp_cache")

param_grid = {
    "clf__alpha": [1e-5, 1e-4, 1e-3],
}

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=3,
    scoring="f1",
    n_jobs=-1
)
grid.fit(X_train, y_train)
print("Best parameters:", grid.best_params_)
print("Best CV score:", grid.best_score_)

# takes 1min30s on 12 cores with 16GB RAM

Best parameters: {'clf__alpha': 1e-05}
Best CV score: 0.6691945452742143


'              precision    recall  f1-score   support\n\n           0       0.84      0.97      0.90     45073\n           1       0.90      0.56      0.69     18742\n\n    accuracy                           0.85     63815\n   macro avg       0.87      0.77      0.80     63815\nweighted avg       0.86      0.85      0.84     63815\n'

|              | precision | recall | f1-score | support |
|--------------|-----------|--------|----------|---------|
| 0            | 0.84      | 0.97   | 0.90     | 45073   |
| 1            | 0.90      | 0.56   | 0.69     | 18742   |
| **accuracy** |           |        | 0.85     | 63815   |
| macro avg    | 0.87      | 0.77   | 0.80     | 63815   |
| weighted avg | 0.86      | 0.85   | 0.84     | 63815   |


In [9]:
# test the model on the validation set
from sklearn.metrics import f1_score
y_val_pred = grid.predict(kaggle_validation['text'])
print(f1_score(kaggle_validation['label'], y_val_pred, average='weighted'))

0.5990685460954345


In [21]:
# predict on the test set

y_test_pred = grid.predict(kaggle_test['text'])
submission = pd.DataFrame({
    'id': kaggle_test.id,
    'label': y_test_pred
})
submission.to_csv('kaggle_submission.csv', index=False)

In [ ]:
# !kaggle competitions submit -c f-25-si-670-kaggle-1 -f kaggle_submission.csv -m "baseline model with Hashing Vectorizer and SGDClassifier"

100%|█████████████████████████████████████████| 534k/534k [00:00<00:00, 932kB/s]
Successfully submitted to F25 SI 670 Kaggle 1